In [378]:
import os
import re
import string
import email
from email.header import decode_header
import csv

In [379]:
no_head_train_path_0 = 'IWSPA-AP-traindata/phish/'
no_head_train_path_1 = 'IWSPA-AP-traindata/legit/'
head_train_phish_0 = 'Dataset_Full_Header_Training/Dataset_Submit_Phish/'
head_train_legit_1 = 'Dataset_Full_Header_Training/Dataset_Submit_Legit/'

In [380]:
# Define the CSV filename
csv_filename = "IWSPA-AP-Parsed.csv"

# Write the extracted information to the CSV file
csv_file = open(csv_filename, mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['email_from', 'email_to', 'email_subject', 'email_body', 'is_phishing'])

58

In [381]:
import email
from email.header import decode_header
from bs4 import BeautifulSoup  # You may need to install the BeautifulSoup library

def get_content(email_content, is_phishing):
    # The email content you provided
    # Create an EmailMessage object from the email content
    count = 0
    try:
        from_address = ''
        to_address = ''
        subject = ''
        msg = email.message_from_string(email_content)
        # print("********\n",msg, "**********\n")
        # Extract the 'From' address
        if ("From" in msg.keys()):
            from_address = decode_header(msg["From"])[0][0]
        
        # Extract the 'To' address
        if ("To" in msg.keys()):
            # print(msg["To"])
            to_address = decode_header(msg["To"])[0][0]
        
        # Extract the 'Subject' of the email
        if ('Subject' in msg.keys()):
            subject = decode_header(msg["Subject"])[0][0]
        body = ''
        # Extract the email body
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == "text/plain":
                    # Accumulate plain text parts from all components
                    body += part.get_payload(decode=True).decode(part.get_content_charset() or 'windows-1251')
                elif part.get_content_type() == "text/html":
                    # If an HTML part is found, parse it and extract the text
                    soup = BeautifulSoup(part.get_payload(decode=True).decode(part.get_content_charset() or 'windows-1251'), "html.parser")
                    body += soup.get_text()
        
        else:
            # If not multipart, directly extract the body
            # print(msg.get_content_charset())
            body = msg.get_payload(decode=True).decode('windows-1251')
    
        body = body.replace('\n', ' ')
        body = ' '.join(body.split())
        csv_writer.writerow([from_address, to_address, subject, body, is_phishing])
        count+=1
    except Exception as e:
        print(e)

    return count
    # print("From:", from_address)
    # print("To:", to_address)
    # print("Subject:", subject)
    # print("Body:", body)

In [382]:
def get_data(path, is_phishing):
    # text_list = list()
    files = os.listdir(path)
    # print(len(files))
    count_processed_data = 0
    for text_file in files:
        file_path = os.path.join(path, text_file)
        read_file = open(file_path,'r+')
        read_text = read_file.read()
        read_file.close()
        # cleaned_text = clean_text(read_text)
        # print(text_file)
        count_processed_data+=get_content(read_text, is_phishing)
    return count_processed_data
        # text_list.append(cleaned_text)

In [383]:
get_data(head_train_phish_0, 1)

503

In [384]:
get_data(head_train_legit_1, 0)

'charmap' codec can't decode byte 0x98 in position 1208: character maps to <undefined>


4081

In [385]:
get_data(no_head_train_path_0, 1)

629

In [386]:
get_data(no_head_train_path_1, 0)

5092